<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Применение-BERT" data-toc-modified-id="Применение-BERT-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Применение BERT</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Обучим модель классификации комментариев на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**План выполнения проекта**

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.

Для выполнения проекта применим предобученную модель *BERT*.

## Подготовка

In [4]:
import numpy as np
import pandas as pd

import time
from tqdm import tqdm
from collections import Counter
from sklearn.metrics import classification_report

import torch
import transformers
import stop_words
import re
from pymystem3 import Mystem
from nltk.stem.snowball import SnowballStemmer

from sklearn.model_selection import train_test_split
from fast_ml.model_development import train_valid_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [5]:
data = pd.read_csv('toxic_comments.csv')
df = data.copy()
display(df.head(), df.shape, df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


(159571, 2)

None

Импортировали необходимые библиотеки и загрузили датасет. Теперь взглянем на предварительный подсчет классов:

In [6]:
Counter(df.toxic)

Counter({0: 143346, 1: 16225})

Мы определили дисбаланс, теперь знаем, что в дальнейшем на это необходимо будет обратить внимание.

Начнем предобработку, применив стемминг для удаления концовки слов. Это альтернатива лемматизации с уменьшенным временем обработки. Также, напишем функцию `clean_text` с помощью библиотеки `re` для очистки текста от лишних символов, пробелов и прочего:

In [7]:
stemmer = SnowballStemmer('english')
stop_words_en = stop_words.get_stop_words('english')

In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\sa-zA-Z0-9@\[\]]',' ',text)
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text)
    text = re.sub('\s{2,}', " ", text)
    new_txt = ''
    for t in text.split(' '):
        if len(t) > 0:
            new_txt = new_txt + stemmer.stem(t) + ' '
    return new_txt[:-1]

Разделим данные на обучающую, валидационную и тестовую выборки, указав параметр `stratify`, как решение для борьбы с дисбалансом:

In [9]:
state = np.random.RandomState(12345)
target = 'toxic'

df_train, df_tmp = train_test_split(df, test_size=0.2, stratify=df[target], random_state=state)
df_valid, df_test = train_test_split(df_tmp, train_size=0.5, stratify=df_tmp[target], random_state=state)
X_train, X_valid, X_test = (df_train['text'], 
                            df_valid['text'],
                            df_test['text'],
                           )

y_train, y_valid, y_test = df_train[target], df_valid[target], df_test[target]


display(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape)

(127656,)

(127656,)

(15957,)

(15957,)

(15958,)

(15958,)

Последней необходимой операцией в рамках подготовки будет написание функций добавления в списки необходимых для нашего последующего сравнения моделей метрик, а также пайплайн модели с применением векторайзера, для преобразования текста в векторы, функции очистки текста и непосредственно модели:

In [10]:
model = []
f1 = []
train_time = []
predict_time = []

def list_append(model_name, metric, time_1, time_2):
    model.append(model_name)
    f1.append(metric)
    train_time.append(time_1)
    predict_time.append(time_2)

In [ ]:
def pipe_model(name, model):
    timer_begin = time.time()

    pipe = Pipeline([
                    ('vectorize', TfidfVectorizer(max_features=10000,
                                   preprocessor=clean_text,
                                   stop_words=stop_words_en)),
                    ('model', model)
    ])
    pipe.fit(X_train, y_train)

    timer_end = time.time()
    train_timer = timer_end - timer_begin

    timer_begin = time.time()

    pred = pipe.predict(X_valid)

    f1 = f1_score(y_valid, pred)
    
    timer_end = time.time()
    predict_timer = timer_end - timer_begin
    
    list_append(name, f1, train_timer, predict_timer)
    print('Затраченное время: %f ms' % train_timer)
    print('Затраченное время: %f ms' % predict_timer)
    print('f1_score: ', f1)
    
    return pipe

А также напишем функцию для поиска оптимальных параметров моделей:

In [ ]:
def search_params(model, params):
    
    timer_begin = time.time()
    
    vect = TfidfVectorizer(max_features=10000,
                           preprocessor=clean_text,
                           stop_words=stop_words_en)
    X = vect.fit_transform(X_train)

    searchCV = RandomizedSearchCV(model, params, scoring='f1', cv=3)  
    search = searchCV.fit(X, y_train)
    
    timer_end = time.time()
    predict_timer = timer_end - timer_begin
    print(predict_timer)
    print(search.best_score_)
    return search.best_params_

**Вывод:** в рамках подготовки мы загрузили данные, обработали их для корректного обучения моделей и написали пайплайн, в который будем загружать модели. Можно переходить к обучению.

## Обучение

Для выбора наилучшей модели будем сравнивать логистическую регрессию, случайный лес, LGBMClassifier, XGBClassifier и модель стохастического градиентного спуска SGDClassifier.

Предварительно будем подбирать оптимальные параметры методом `RandomizedSearchCV`:

In [ ]:
lr_params = {'penalty': ['l1','l2'], 
             'C': [1, 10, 100, 1000]}
search_params(LogisticRegression(), lr_params)

136.74245381355286
0.7741688366843018


{'penalty': 'l2', 'C': 10}

In [ ]:
lr_model = LogisticRegression(penalty='l2', C=10, random_state=state)
pipe_model('LogisticRegression', lr_model)

Затраченное время: 98.604325 ms
Затраченное время: 12.949096 ms
f1_score:  0.7828178694158074


Pipeline(steps=[('vectorize',
                 TfidfVectorizer(max_features=10000,
                                 preprocessor=<function clean_text at 0x00000213428C0670>,
                                 stop_words=['a', 'about', 'above', 'after',
                                             'again', 'against', 'all', 'am',
                                             'an', 'and', 'any', 'are',
                                             "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', "can't",
                                             'cannot', 'could', "couldn't",
                                             'did', ...])),
                ('model',
                 LogisticRegression(C=10,
                                    random_state=RandomState(MT19937) at 0x2134284D740))])

In [ ]:
rfc_params = {'n_estimators': [100, 200, 300],
              'max_depth': [10, 20, 30],
              'min_samples_split': [2, 4, 6],
              'min_samples_leaf': [1, 2, 4]}
search_params(RandomForestClassifier(random_state=state), rfc_params)

773.6074695587158
0.24853330065676063


{'n_estimators': 300,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_depth': 30}

In [ ]:
rfc = RandomForestClassifier(random_state=state, n_estimators=300, min_samples_split=2, min_samples_leaf=1, max_depth=30)
pipe_model('RandomForestClassifier', rfc)

Затраченное время: 145.653963 ms
Затраченное время: 12.535712 ms
f1_score:  0.2340772999455634


Pipeline(steps=[('vectorize',
                 TfidfVectorizer(max_features=10000,
                                 preprocessor=<function clean_text at 0x00000213428C0670>,
                                 stop_words=['a', 'about', 'above', 'after',
                                             'again', 'against', 'all', 'am',
                                             'an', 'and', 'any', 'are',
                                             "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', "can't",
                                             'cannot', 'could', "couldn't",
                                             'did', ...])),
                ('model',
                 RandomForestClassifier(max_depth=30, n_estimators=300,
                                        random_state=RandomState(MT19

In [ ]:
lgbm_params = {
    'n_estimators': [400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [9, 12, 15],
    'num_leaves': [25, 50, 75],
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20]
}
search_params(LGBMClassifier(), lgbm_params)

2809.944193840027
0.7734861990499526


{'subsample_freq': 20,
 'subsample': 0.8,
 'num_leaves': 25,
 'n_estimators': 700,
 'min_split_gain': 0.4,
 'max_depth': 12,
 'colsample_bytree': 0.7}

In [ ]:
lgbm = LGBMClassifier(max_depth=12, 
                      n_estimators=700, 
                      random_state=state, 
                      n_jobs=-1, 
                      subsample=0.8, 
                      subsample_freq=20, 
                      min_split_gain=0.4,
                      num_leaves=25,
                      colsample_bytree=0.7)
pipe_model('LGBMClassifier', lgbm)

Затраченное время: 173.974717 ms
Затраченное время: 13.843389 ms
f1_score:  0.7784765897973446


Pipeline(steps=[('vectorize',
                 TfidfVectorizer(max_features=10000,
                                 preprocessor=<function clean_text at 0x00000213428C0670>,
                                 stop_words=['a', 'about', 'above', 'after',
                                             'again', 'against', 'all', 'am',
                                             'an', 'and', 'any', 'are',
                                             "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', "can't",
                                             'cannot', 'could', "couldn't",
                                             'did', ...])),
                ('model',
                 LGBMClassifier(colsample_bytree=0.7, max_depth=12,
                                min_split_gain=0.4, n_estimators=700,
   

In [ ]:
sgd_params = {
       'alpha' : np.linspace(0.00001, 0.0001, 15),
       'learning_rate': ['optimal', 'constant', 'invscaling'],
       'eta0' : np.linspace(0.00001, 0.0001, 15),
       'max_iter' : np.arange(5,10),
             }

search_params(SGDClassifier(), sgd_params)

129.892676115036
0.7322868295635584


{'max_iter': 7,
 'learning_rate': 'optimal',
 'eta0': 6.785714285714286e-05,
 'alpha': 3.571428571428572e-05}

In [ ]:
sgd = SGDClassifier(learning_rate='optimal', 
                    max_iter=7, 
                    random_state=state, 
                    alpha=3.571428571428572e-05, 
                    eta0=6.785714285714286e-05)
pipe_model('SGDClassifier', sgd)

Затраченное время: 92.354152 ms
Затраченное время: 11.989958 ms
f1_score:  0.7323518308795771


Pipeline(steps=[('vectorize',
                 TfidfVectorizer(max_features=10000,
                                 preprocessor=<function clean_text at 0x00000213428C0670>,
                                 stop_words=['a', 'about', 'above', 'after',
                                             'again', 'against', 'all', 'am',
                                             'an', 'and', 'any', 'are',
                                             "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', "can't",
                                             'cannot', 'could', "couldn't",
                                             'did', ...])),
                ('model',
                 SGDClassifier(alpha=3.571428571428572e-05,
                               eta0=6.785714285714286e-05, max_iter=7,
          

In [ ]:
xgb_params = {
              'learning_rate': [0.1, 0.3], 
              'max_depth': [7, 9, 10],
              'min_child_weight': [5, 10, 15],
              'colsample_bytree': [0.8, 0.9, 1],
              'n_estimators': [600, 800, 1000]
              }
search_params(XGBClassifier(random_state=state), xgb_params)

5781.546944856644
0.7580207957347523


{'n_estimators': 800,
 'min_child_weight': 5,
 'max_depth': 10,
 'learning_rate': 0.1,
 'colsample_bytree': 0.9}

In [ ]:
xgb = XGBClassifier(random_state=state, 
                    n_estimators=800,
                    min_child_weight=5,
                    max_depth=10,
                    learning_rate=0.1,
                    colsample_bytree=0.9)

pipe_model('XGBClassifier', xgb)

Затраченное время: 469.374887 ms
Затраченное время: 14.462236 ms
f1_score:  0.7667844522968197


Pipeline(steps=[('vectorize',
                 TfidfVectorizer(max_features=10000,
                                 preprocessor=<function clean_text at 0x00000213428C0670>,
                                 stop_words=['a', 'about', 'above', 'after',
                                             'again', 'against', 'all', 'am',
                                             'an', 'and', 'any', 'are',
                                             "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', "can't",
                                             'cannot', 'could', "couldn't",
                                             'did', ...]...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
          

**Вывод:** модели обучены, результаты их работы записаны, теперь можно провести анализ и определить оптимальную по результату их работы на валидационной выборке.

### Применение BERT

В целях экономии времени возьмем часть данных:

In [11]:
sample = data.sample(50000, random_state=state)
Counter(sample.toxic)

Counter({0: 44861, 1: 5139})

Инициализируем `BertTokenizer` и `BertModel`, предобученные на токсичных данных `unitary/toxic-bert`:

In [13]:
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')
model_bert = transformers.BertModel.from_pretrained('unitary/toxic-bert') 

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Токинизируем исходные данные, указав максимальную размерность:

In [14]:
tqdm.pandas()
tokenized = sample['text'].progress_apply((lambda x: tokenizer.encode(
                                            x, max_length=512, truncation=True, add_special_tokens=True)))

100%|██████████| 50000/50000 [01:41<00:00, 490.32it/s]


In [15]:
tokenized

81198     [101, 2339, 2038, 1996, 4431, 4957, 2000, 2601...
155543    [101, 3335, 9587, 29519, 7065, 8743, 1045, 100...
115179    [101, 1000, 2009, 2038, 2036, 2042, 4937, 2026...
47125     [101, 1037, 1012, 1041, 1012, 7570, 2271, 2386...
157197    [101, 2310, 4359, 1996, 3860, 2006, 2023, 2396...
                                ...                        
157659    [101, 9353, 4168, 2386, 9353, 4168, 2386, 2003...
138286    [101, 10047, 2260, 1998, 1045, 2064, 3305, 200...
19535     [101, 2002, 2038, 2363, 3278, 6325, 1999, 2248...
38398     [101, 1045, 2228, 2017, 2323, 4895, 23467, 203...
36707     [101, 1000, 12831, 4234, 999, 3963, 2361, 2595...
Name: text, Length: 50000, dtype: object

Применим `padding` для уравнивания длин всех векторов по самому большому вектору:

In [16]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

(50000, 512)

Теперь "накинем" маску на не значимые значения векторов:

In [17]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(50000, 512)

Преобразуем данные в формат тензоров — многомерных векторов в библиотеке `torch`:

In [18]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

In [19]:
# Код для вычисления доступной памяти про работе с Colab:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()

gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=28ac5469c771ef87cff92b01bc3fec02d8f1bdf1408038c0817255b41f7830f3
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gen RAM Free: 10.8 GB  |     Proc size: 2.1 GB
GPU RAM Free: 15106MB | Used: 3MB | Util   0% | Total     15109MB


Наконец, создадим эмбединги, передав модели данные и маску:

In [20]:
%%time
from tqdm import notebook
batch_size = 100
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda()
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

        with torch.no_grad():
            model_bert.cuda()
            batch_embeddings = model_bert(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        del attention_mask_batch
        del batch_embeddings

features = np.concatenate(embeddings) 

  0%|          | 0/500 [00:00<?, ?it/s]

CPU times: user 30min 16s, sys: 4.09 s, total: 30min 20s
Wall time: 31min 25s


Создав признаки, поделим данные на обучающую и тестовую выборки:

In [21]:
target = sample['toxic']
train_features, test_features, train_target, test_target = train_test_split(
features, target, test_size=0.15)

Признаки и ответы есть, возьмем навскидку модель LightBoost и подберем параметры на наших относительно новых данных:

In [22]:
lgbm_params = {
    'n_estimators': [400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [9, 12, 15],
    'num_leaves': [25, 50, 75],
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20]
}
searchCV = RandomizedSearchCV(LGBMClassifier(), lgbm_params, scoring='f1', cv=4)  
search = searchCV.fit(train_features, train_target)
best_model = search.best_estimator_
print(search.best_score_)

0.9436528540345179


In [26]:
timer_begin = time.time()

best_model.fit(train_features, train_target)

timer_end = time.time()
train_timer = timer_end - timer_begin
timer_begin = time.time()

bert_pred = best_model.predict(test_features)
bert_score = f1_score(test_target, bert_pred)

timer_end = time.time()
predict_timer = timer_end - timer_begin

print('Затраченное время: %f ms' % train_timer)
print('Затраченное время: %f ms' % predict_timer)
print('f1_score: ', bert_score)

Затраченное время: 57.927689 ms
Затраченное время: 0.220954 ms
f1_score:  0.9087193460490463


**Вывод:** получение эмбедингов и подбор параметров занимаем достаточно много времени (около 30 минут для каждый процедуры), однако модель обучается и предсказывает токсичность относительно быстро при очень высокой f1 - 0.90 . Все готово к итоговому выводу.

## Выводы

In [ ]:
list_tuples = list(zip(model, f1, train_time, predict_time))
frame = pd.DataFrame(list_tuples, columns=['model', 'f1', 'train_time, sec', 'predict_time, sec'])
frame.set_index('model')

,f1,"train_time, sec","predict_time, sec"
model,,,
LogisticRegression,0.782818,98.604325,12.949096
RandomForestClassifier,0.234077,145.653963,12.535712
LGBMClassifier,0.778477,173.974717,13.843389
SGDClassifier,0.732352,92.354152,11.989958
XGBClassifier,0.766784,469.374887,14.462236


**Вывод:** что касается классических методов машинного обучения, наилучший результат, как ни странно показала простая логистическая регрессия с подобранными параметрами - 0.78. Ее и будем использовать для тестовой выборки.

In [ ]:
timer_begin = time.time()

pipe = Pipeline([
                    ('vectorize', TfidfVectorizer(max_features=10000,
                                   preprocessor=clean_text,
                                   stop_words=stop_words_en)),
                    ('model', lr_model)
    ])
pipe.fit(X_train, y_train)

timer_end = time.time()
train_timer = timer_end - timer_begin

timer_begin = time.time()

pred = pipe.predict(X_test)

f1 = f1_score(y_test, pred)
    
timer_end = time.time()
predict_timer = timer_end - timer_begin

print('Затраченное время: %f ms' % train_timer)
print('Затраченное время: %f ms' % predict_timer)
print('f1_score: ', f1)

Затраченное время: 95.281978 ms
Затраченное время: 11.871245 ms
f1_score:  0.7982396750169263


**Вывод:** поставленная задача в достижении результата F1-метрики не ниже 0.75 достигнула. Полученная применением логической регрессии метрика составила - 0.798. Однако, предобученная нейронная сеть `Bert` показывает очень высокий результат в 0.90 F1 score. 

Итоговая рекомендация: если есть возможность и средства применять нейросеть, то однозначно, `Bert`, предобученный на токсичных данных - наилучший вариант. Если же обращаться к классическим методам машинного обучения, то логистическая регрессия также справляется с поставленной задачей (F1 > 0.75), но ее показатели на примерно 0.10 ниже, что представляет собой существенную разницу.